In [1]:
!pip install torchinfo
!pip install optuna

import optuna
import glob
import re
import numpy as np
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import copy
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from torchinfo import summary
from PIL import Image
from collections import Counter
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True


In [2]:
!git clone https://github.com/Bjarten/early-stopping-pytorch/
%cd ./early-stopping-pytorch/
!python pytorchtools.py install
from pytorchtools import EarlyStopping

Cloning into 'early-stopping-pytorch'...
remote: Enumerating objects: 92, done.
remote: Total 92 (delta 0), reused 0 (delta 0), pack-reused 92
Unpacking objects: 100% (92/92), done.
/kaggle/working/early-stopping-pytorch


In [3]:
%cd ../

/kaggle/working


In [4]:
def img_files(path):  
  img=[]
  tag=[]
  for file in sorted(glob.glob(path)):
    img.append(file)
    s=re.match("^.*?-",file).group()
    if s in tag:
      continue
    else:
      tag.append(s)

  
  return img,tag

In [5]:
def reset_weights(m):
    if isinstance(m, nn.LSTM) or isinstance(m, nn.Linear):
        m.reset_parameters()

In [6]:
def preprocess_data(data):
  l=[]
  data=np.reshape(np.array(data),(-1,20))
  for i in data:
    s=re.search("(?<=-).?",i[0])
    if s.group()=='N':
      l.append(0)
    else:
      l.append(1)
  
  label=np.array(l,dtype=np.float32)

  return data,label

In [7]:
class RoboDataset(Dataset):
  def __init__(self, file,labels,transform):
    self.imagefiles=file
    self.labels=labels
    self.transform=transform
  def __len__(self):
    return len(self.imagefiles)
  def __getitem__(self,idx):
    l=[]
    for i in self.imagefiles[idx]:
      img=Image.open(i)
      l.append(img)
    frame=[]
    for j in l:
      j=self.transform(j)
      frame.append(j)
    frame=torch.stack(frame)

    return frame,self.labels[idx]

In [8]:
class LSTM(nn.Module):
  def __init__(self,hidden_size,num_layers,num_classes):
    super(LSTM, self).__init__()
    
    self.hidden_size=hidden_size
    self.num_layers=num_layers
    self.num_classes=num_classes
    self.backbone = torch.hub.load('/kaggle/working/trainval/yolov5/yolov5/', 'custom', source='local', path='/kaggle/working/trainval/yolov5n_backbone.pt', force_reload=True, autoshape=False)
    features=256
    for param in self.backbone.parameters():
      param.requires_grad = False
    self.pool=nn.AdaptiveAvgPool2d(1)
    self.flat=nn.Flatten()
    self.lstm = nn.LSTM(input_size=features, hidden_size=hidden_size,num_layers=num_layers, batch_first=True)    
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    batch, seq, C,H, W = x.size()
    c_in = x.view(batch * seq, C, H, W)
    bone_out = self.backbone(c_in)
    pool_out=self.pool(bone_out)
    flat_out=torch.flatten(pool_out)
    r_out, (h_n, h_c) = self.lstm(flat_out.view(batch,seq,-1))
    out = r_out[:, -1, :]
    out=self.fc(out)
    return out

In [9]:
def train_model(modelo,earlystop,epochs,trainloader,validloader,optimizer,criterion):

  early_stopping = EarlyStopping(patience=earlystop)
  history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}
  for i in range(1,epochs):
    modelo.train()
    t_loss=0
    t_correct=0
    t_total=0
  
    for j,(inputs,labels) in enumerate(trainloader):
      inputs=inputs.to(device)
      labels=labels.unsqueeze(1).to(device)
      
      optimizer.zero_grad()
      outputs=modelo(inputs)
      loss=criterion(outputs,labels)
      pred=(outputs>0.0).float()

      loss.backward()
      optimizer.step()

      t_loss += loss.item()
      t_correct+=(pred==labels).sum().item()
      t_total+=labels.size(0)

    train_loss=t_loss/len(trainloader)
    tr_acc=100.*t_correct/t_total


    print("Epoch: %d Training, loss: %1.5f, correct/total: %d / %d, accuracy:%1.3f " % (i, train_loss,t_correct,t_total,tr_acc))
      
    modelo.eval()
    v_loss=0
    v_correct=0
    v_total=0

    with torch.no_grad():
      for inputs,labels in validloader:
        inputs=inputs.to(device)
        labels=labels.unsqueeze(1).to(device) 
      
        outputs=modelo(inputs)

        loss=criterion(outputs,labels)
        pred=(outputs>0.0).float()

        v_correct+=(pred==labels).sum().item()
        v_loss += loss.item()
        v_total+=labels.size(0)

    val_loss=v_loss/len(validloader)
    v_acc=100.*v_correct/v_total


        
    print("Epoch: %d Validation, loss: %1.5f, correct/total: %d / %d, accuracy:%1.3f " % (i, val_loss,v_correct,v_total,v_acc))
    history['train_loss'].append(train_loss)
    history['test_loss'].append(val_loss)
    history['train_acc'].append(tr_acc)
    history['test_acc'].append(v_acc)
    early_stopping(val_loss, modelo)
          
    if early_stopping.early_stop:
        print("Early stopping")
        break
        
  return modelo,history

In [10]:
from distutils.dir_util import copy_tree
copy_tree('/kaggle/input/', '/kaggle/working')

['/kaggle/working/trainval/trainval/trainval/V275-R0761.png',
 '/kaggle/working/trainval/trainval/trainval/V685-R5045.png',
 '/kaggle/working/trainval/trainval/trainval/V340-N20351.png',
 '/kaggle/working/trainval/trainval/trainval/V104-N2101.png',
 '/kaggle/working/trainval/trainval/trainval/V850-N0301.png',
 '/kaggle/working/trainval/trainval/trainval/V295-R1140.png',
 '/kaggle/working/trainval/trainval/trainval/V191-R0476.png',
 '/kaggle/working/trainval/trainval/trainval/V334-N0610.png',
 '/kaggle/working/trainval/trainval/trainval/V691-R0490.png',
 '/kaggle/working/trainval/trainval/trainval/V478-N1241.png',
 '/kaggle/working/trainval/trainval/trainval/V346-N0721.png',
 '/kaggle/working/trainval/trainval/trainval/V810-N0276.png',
 '/kaggle/working/trainval/trainval/trainval/V397-R0685.png',
 '/kaggle/working/trainval/trainval/trainval/V736-N0121.png',
 '/kaggle/working/trainval/trainval/trainval/V552-N0276.png',
 '/kaggle/working/trainval/trainval/trainval/V143-R3800.png',
 '/kagg

In [11]:
train_files,tags=img_files("/kaggle/working/trainval/trainval/trainval/*.png")
trainX,trainY=preprocess_data(train_files)

In [12]:
train_transform=transforms.Compose([
      transforms.Resize((320,320)),
      transforms.ToTensor(),  
      transforms.Normalize([0.4335, 0.4272, 0.4201], [0.2497, 0.2502, 0.2524]),
                            
])

valid_transform=transforms.Compose([
      transforms.Resize((320,320)),
      transforms.ToTensor(),  
      transforms.Normalize([0.4335, 0.4272, 0.4201], [0.2497, 0.2502, 0.2524]),
                           
])

In [13]:
def hyptuning(version):
    param={"num_layers":version.suggest_int("num_layer",1,4),
           "hidden_size":version.suggest_int("hidden_size",16,128),
           "learning_rate":version.suggest_uniform("learning_rate",5e-6,1e-2),
           

        
    }
    epochs=30
    hidden_size=param["hidden_size"]
    num_layers=param["num_layers"]
    num_classes=1
    learning_rate=param["learning_rate"]
    earlystop=5

    modelo = LSTM(hidden_size, num_layers,num_classes)
    modelo.to(device)
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(modelo.parameters(), lr=param["learning_rate"])
    prueba=RoboDataset(trainX,trainY,transform=train_transform)
    nfold=8
    kfold=KFold(n_splits=nfold,shuffle=True)
    foldperf={}
    for fold,(train_idx,test_idx) in enumerate(kfold.split(prueba)):
        print('Fold No. {}'.format(fold+1))
        train_sampler=SubsetRandomSampler(train_idx)
        test_sampler=SubsetRandomSampler(test_idx)
        trainloader=DataLoader(prueba,batch_size=64,sampler=train_sampler)
        validloader=DataLoader(prueba,batch_size=64,sampler=test_sampler)
        modelo,history=train_model(modelo,earlystop,epochs,trainloader,validloader,optimizer,criterion)
        foldperf['fold{}'.format(fold+1)] = history  
        modelo.apply(reset_weights)
    
    testloss_fold=[]
    k=8
    for f in range(k):
        testloss_fold.append(np.min(foldperf['fold{}'.format(f+1)]['test_loss']))
            

    return np.mean(testloss_fold)

In [14]:
study=optuna.create_study(direction="minimize")
study.optimize(hyptuning,n_trials=3)
print("Mejor resultado")
versions=study.best_trial
print(versions.values)
print(versions.params)

[I 2022-05-16 13:48:13,339] A new study created in memory with name: no-name-d372d1c7-85ec-4073-b540-0f23973faa42


Fold No. 1
Epoch: 1 Training, loss: 0.68335, correct/total: 398 / 692, accuracy:57.514 
Epoch: 1 Validation, loss: 0.66505, correct/total: 66 / 99, accuracy:66.667 
Epoch: 2 Training, loss: 0.67057, correct/total: 420 / 692, accuracy:60.694 
Epoch: 2 Validation, loss: 0.73309, correct/total: 51 / 99, accuracy:51.515 
EarlyStopping counter: 1 out of 5
Epoch: 3 Training, loss: 0.64468, correct/total: 447 / 692, accuracy:64.595 
Epoch: 3 Validation, loss: 0.65648, correct/total: 67 / 99, accuracy:67.677 
Epoch: 4 Training, loss: 0.60001, correct/total: 485 / 692, accuracy:70.087 
Epoch: 4 Validation, loss: 0.60858, correct/total: 65 / 99, accuracy:65.657 
Epoch: 5 Training, loss: 0.56547, correct/total: 497 / 692, accuracy:71.821 
Epoch: 5 Validation, loss: 0.58720, correct/total: 68 / 99, accuracy:68.687 
Epoch: 6 Training, loss: 0.55132, correct/total: 508 / 692, accuracy:73.410 
Epoch: 6 Validation, loss: 0.62187, correct/total: 71 / 99, accuracy:71.717 
EarlyStopping counter: 1 out of

[I 2022-05-16 17:04:32,107] Trial 0 finished with value: 0.5631208829581738 and parameters: {'num_layer': 3, 'hidden_size': 89, 'learning_rate': 0.003117221677836792}. Best is trial 0 with value: 0.5631208829581738.


Epoch: 29 Validation, loss: 0.59919, correct/total: 71 / 98, accuracy:72.449 
EarlyStopping counter: 2 out of 5
Fold No. 1
Epoch: 1 Training, loss: 0.69349, correct/total: 403 / 692, accuracy:58.237 
Epoch: 1 Validation, loss: 0.68266, correct/total: 61 / 99, accuracy:61.616 
Epoch: 2 Training, loss: 0.67759, correct/total: 401 / 692, accuracy:57.948 
Epoch: 2 Validation, loss: 0.69660, correct/total: 48 / 99, accuracy:48.485 
EarlyStopping counter: 1 out of 5
Epoch: 3 Training, loss: 0.66982, correct/total: 420 / 692, accuracy:60.694 
Epoch: 3 Validation, loss: 0.62510, correct/total: 63 / 99, accuracy:63.636 
Epoch: 4 Training, loss: 0.61033, correct/total: 469 / 692, accuracy:67.775 
Epoch: 4 Validation, loss: 0.59114, correct/total: 68 / 99, accuracy:68.687 
Epoch: 5 Training, loss: 0.57388, correct/total: 488 / 692, accuracy:70.520 
Epoch: 5 Validation, loss: 0.60792, correct/total: 66 / 99, accuracy:66.667 
EarlyStopping counter: 1 out of 5
Epoch: 6 Training, loss: 0.55785, corre

[I 2022-05-16 20:21:09,981] Trial 1 finished with value: 0.6007155738770962 and parameters: {'num_layer': 3, 'hidden_size': 84, 'learning_rate': 0.006435555948666753}. Best is trial 0 with value: 0.5631208829581738.


Epoch: 11 Validation, loss: 0.69186, correct/total: 51 / 98, accuracy:52.041 
EarlyStopping counter: 5 out of 5
Early stopping
Fold No. 1
Epoch: 1 Training, loss: 0.68470, correct/total: 413 / 692, accuracy:59.682 
Epoch: 1 Validation, loss: 0.68884, correct/total: 51 / 99, accuracy:51.515 
Epoch: 2 Training, loss: 0.68236, correct/total: 413 / 692, accuracy:59.682 
Epoch: 2 Validation, loss: 0.69694, correct/total: 51 / 99, accuracy:51.515 
EarlyStopping counter: 1 out of 5
Epoch: 3 Training, loss: 0.68074, correct/total: 413 / 692, accuracy:59.682 
Epoch: 3 Validation, loss: 0.69011, correct/total: 51 / 99, accuracy:51.515 
EarlyStopping counter: 2 out of 5
Epoch: 4 Training, loss: 0.67908, correct/total: 413 / 692, accuracy:59.682 
Epoch: 4 Validation, loss: 0.68866, correct/total: 51 / 99, accuracy:51.515 
Epoch: 5 Training, loss: 0.67670, correct/total: 413 / 692, accuracy:59.682 
Epoch: 5 Validation, loss: 0.69437, correct/total: 51 / 99, accuracy:51.515 
EarlyStopping counter: 1

[I 2022-05-17 01:20:40,945] Trial 2 finished with value: 0.6433266140520573 and parameters: {'num_layer': 2, 'hidden_size': 82, 'learning_rate': 3.745250410544068e-05}. Best is trial 0 with value: 0.5631208829581738.


Epoch: 20 Validation, loss: 0.66728, correct/total: 64 / 98, accuracy:65.306 
EarlyStopping counter: 5 out of 5
Early stopping
Mejor resultado
[0.5631208829581738]
{'num_layer': 3, 'hidden_size': 89, 'learning_rate': 0.003117221677836792}
